# Explicabilidad en Inteligencia Artificial

## 1. Introducción

Debido al incremento exponencial del desarrollo y la integración de sistemas inteligentes en actividades de diversa naturaleza y frecuencia, la comprensión de su funcionamiento resulta fundamental por diversos motivos. 
En primer lugar si un modelo pretende **desempeñar un papel fundamental como una herramienta para ayudar a la toma de decisiones**, su modo de razonamiento debe ser fácilmente entendible por un ser humano tradicional. Así se puede analizar su **comportamiento** para observar rasgos indeseados que poder corregir, asegurar que sus pilares se encuentran alineados con los objetivos del problema que aborda, que cumple una determinada **ética** establecida en torno a las soluciones que propone, además de medidas de seguridad críticas con las que controlar la **calidad** de las mismas. 

De este modo en este notebook se pretende aplicar distintas técnicas de explicabilidad con la que comprender más detalladamente la conducta característica de los clasificadores elaborados para la detección de textos sexistas.

### 1.1. Tipos de explicabilidad

* **Modelos agnósticos locales**. Esta categoría de técnicas se pueden emplear sobre cualquier tipo de modelo de caja negra aunque únicamente se centran en una sola muestra.

## 2. Estructura del notebook

1. Introducción a la Explicabilidad en Inteligencia Artificial
2. Estructura del notebook
3. Instalación y carga de librerías
4. Lectura y carga de datos procesados
5. Técnicas de explicabilidad
6. Conclusiones

## 3. Instalación y carga de librerías

Este apartado tiene como único propósito cargar las librerías y dependencias necesarias para la ejecución de este notebook, así como las funciones propiamente desarrolladas. Previo a ello deberán ser instaladas bien ejecutando el script *setup.sh* mediante el comando `bash setup.sh` con permisos de ejecución en distribuciones Linux, o bien ejecutando el compando `pip install -r requirements.txt`.

In [1]:
%%capture
import sys
sys.path.append('../scripts')

# Import encoding functions
from encoding import *

# Import functions to generate synthetic texts
from data_augmentation import create_new_texts_by_removing_words, generate_text_using_gpt3

# pandas: to read the datasets
import pandas as pd

# numpy: to work with predicted probabilities
import numpy as np

# skelearn: to encode documents
from sklearn.feature_extraction.text import CountVectorizer

# pickle: to load Logistic Regression models
import pickle 

# keras: to load pre-trained models
from keras.utils import pad_sequences
from keras.models import load_model
from keras.preprocessing.text import Tokenizer

# tensorflow / transformers: to work with pre-trained BERT models
import tensorflow as tf
from transformers import *

# lime: to create local explanations
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

# random: to choose random samples
from random import randrange

2023-04-27 14:17:37.683327: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 14:17:37.812086: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-27 14:17:37.812104: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-27 14:17:38.471532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## 4. Lectura y carga de datos procesados

En esta sección se pretende leer y cargar los datos de entrenamiento y test ya procesados a los que se les ha aplicado las siguientes técnicas de tratamiento de documentos:

  - Elimina URLs.
  - Elimina usuarios mencionados.
  - Elimina caracteres especiales, no alfabéticos y signos de puntuación.
  - Convierte todos los caracteres en minúsculas.

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.865 muestras**.
* Conjunto de validación: **4.296 muestras**.

In [2]:
# Read already processed EXIST datasets
train_df = pd.read_csv('../data/proc_EXIST2021_train.csv')
test_df = pd.read_csv('../data/proc_EXIST2021_test.csv')

# Show the dimensions of the datasets
print('Train dataset dimensions:', train_df.shape)
print('Test dataset dimensions:', test_df.shape)

Train dataset dimensions: (6865, 6)
Test dataset dimensions: (4296, 6)


## 5. Técnicas de explicabilidad

### 5.1. LIME

Se trata de un procedimiento que genera **explicaciones locales a una única muestra dentro de una determinada clase** cuyo objetivo consiste en determinar la relevancia de cada característica (palabra) proporcionada para su predicción. Para ello genera un **modelo lineal** que se entrena en base a perturbaciones generadas a partir de un ejemplo seleccionado y luego es evaluado dependiendo de la similitud entre las predicciones producidas y la real asociada a la entrada suministrada. Esta tarea es posible mediante el uso de la librería *lime* de Python a la que se le debe proporcionar una muestra de ejemplo junto con una función que permite obtener la predicción de las perturbaciones creadas.

Puesto que se trata de un método local a un ejemplo, se propone su aplicación en los conjuntos de **falsos positivos y negativos** analizados en notebooks anteriores dentro de los mejores modelos obtenidos para cada arquitectura en inglés y español, con la idea de comprobar cuáles son los términos destacables que han liderado hacia una clasificación errónea. Como los volumenes de muestras son bastante considerables, se pretende seleccionar **diez ejemplos aleatoriamente de cada intervalo de confianza** para detectar la existencia de características diferentes en función de la seguridad demostrada por parte del modelo al identificar cada dato de test.

#### 5.1.1. Modelo inglés de Regresión Logística

Comenzamos ejecutando este método sobre el mejor clasificador inglés de Regresión Logística encontrado cuya codificación se realizó transformando los textos en bolsas de palabras. A continuación se observa que este modelo se caracteriza por tener 213 falsos positivos y 438 falsos negativos, unas cifras de errores demasiado elevadas para considerarlo como una solución de calidad.

In [41]:
# Load a Logistic Regression model specialized in English
en_lr_model = pickle.load(open('../models/multileng_lr_models/en_bag_words_model.sav', 'rb'))

# Filter the train documents by language
en_train_df = train_df[train_df['language'] == 'en']
en_train_texts = list(en_train_df['clean_text'].values)

# Filter the test documents by language
en_test_df = test_df[test_df['language'] == 'en']
en_test_texts = list(en_test_df['clean_text'].values)

# Convert train and test documents to bag of words
en_train_bag_words, en_test_bag_words = to_bag_of_words(
    train_docs=en_train_texts, 
    test_docs=en_test_texts)

# Predict over the test dataset
en_test_df['task1_pred_classes'] = en_lr_model.predict(X=en_test_bag_words)
en_test_df['task1_pred_probs'] = [max(prob) for prob in en_lr_model.predict_proba(X=en_test_bag_words)]

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

No. of false negatives: 438
No. of false positives: 213



Las conclusiones extraídas de la generación de explicaciones locales a un subconjunto de **falsos negativos** se presentan a continuación:

* Los falsos negativos pertenecientes al más elevado intervalo de confianza otorga un grado de **no-sexismo a palabras sin significado** como pronombres, determinantes, preposiciones y verbos auxiliares, que pueden contrarrestar el efecto de terminología sexista. 

* Aquellos cometidos bajo un rango de confianza alto demuestran un notable grado de **ironía** con terminología positiva aunque con significados profundamente sexistas. Este fenómeno ya se intuía en el análisis exploratorio de datos que se realizó al comienzo de este proyecto.

* Finalmente los falsos negativos que se encuentran dentro del intervalo moderado de confianza están caracterizados por su alta composición de **hashtags y errores ortográficos** que dificultan su análisis e identificación automáticos. No obstante, pese a haber tratado de detectar y corregir los términos gramaticalmente erróneos empleando diversas librerías, ninguna de ellas ha podido conseguir la mejora de la capacidad de predicción de los modelos de Regresión Logística.

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def predict_en_lrm_model(sample: str):

        # Train a CountVectorizer object based on train documents
        bg_vectorizer = CountVectorizer()
        bg_vectorizer.fit(raw_documents=en_train_texts)

        # Encode the provided test sample 
        encoded_sample = bg_vectorizer.transform(raw_documents=sample).toarray()
        
        # Predict the probabilities for both classes
        pred_prob = en_lr_model.predict_proba(X=encoded_sample)
        return np.array(pred_prob)

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

Replicando el mismo procedimiento anterior aunque aplicado a los **falsos positivos** se han podido descubrir las siguientes pesquisas:

* Los falsos positivos de los dos intervalos superiores de confianza **contienen noticias y reivindaciones de usuarios contra los ataques y la violencia contra la mujer**. Como se detalló en el análisis exploratorio de datos, las temáticas del conjunto de test son bien distintas de las tratadas en el dataset de entrenamiento. El problema reside en que términos comunes como "mujeres", "feminismo", etc. son catalogados individualmente como sexistas y como consecuencia producen este comportamiento erróneo en el modelo.

* Los falsos positivos situados en el intervalo moderado de confianza destacan por tratar **otros tópicos** en los que aparecen también palabras de género como "mujeres", "marido", entre otras que pese a ser clasificadas como sexistas su significado no es tal. De nuevo, este hecho fue descubierto durante el análisis exploratorio de datos inicial en el que ya se aventuraba las discrepancias de contenido entre ambos datasets.

#### 5.1.2. Modelo español de Regresión Logística

A continuación se repite el estudio previo pero haciendo uso del mejor modelo español de Regresión Logística con sus correspondientes documentos en dicho idioma. En los siguientes resultados se puede apreciar que existen 462 documentos sexistas no detectados y 199 textos no sexistas erróneamente clasificados.

In [42]:
# Load a Logistic Regression model specialized in Spanish
es_lr_model = pickle.load(open('../models/multileng_lr_models/es_bag_words_model.sav', 'rb'))

# Filter the train documents by language
es_train_df = train_df[train_df['language'] == 'es']
es_train_texts = list(es_train_df['clean_text'].values)

# Filter the test documents by language
es_test_df = test_df[test_df['language'] == 'es']
es_test_texts = list(es_test_df['clean_text'].values)

# Convert train and test documents to bag of words
es_train_bag_words, es_test_bag_words = to_bag_of_words(
    train_docs=es_train_texts, 
    test_docs=es_test_texts)

# Predict over the test dataset
es_test_df['task1_pred_classes'] = es_lr_model.predict(X=es_test_bag_words)
es_test_df['task1_pred_probs'] = [max(prob) for prob in es_lr_model.predict_proba(X=es_test_bag_words)]

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

No. of false negatives: 462
No. of false positives: 199



Las deducciones descubiertas tras la visualización de diez muestras positivas clasificadas erróneamente seleccionadas aleatoriamente por cada intervalo de confianza, como se ha efectuado con el clasificador inglés, han sido idénticamente similares a las observadas previamente.

Por otro lado en relación a los falsos positivos, es decir ejemplos negativos identificados como positivos, además de compartir las características descubiertas anteriormente también cabe destacar la **sexualización por parte del modelo de prendas femeninas**, como la falda, además de la no comprensión de terminología latinoamericana puesto que no se les asigna ninguna clase o valor por lo que se consideran neutros.

#### 5.1.3. Modelo LSTM inglés

En esta sección se cambia de arquitectura para generar exlicaciones locales utilizando el mejor modelo LSTM específico de documentos ingleses compuesto por capas bidireccionales. En los siguientes resultados se puede observar la existencia de más de 398 falsos negativos y 246 falsos positivos. A continuación se procede a muestrear diez ejemplos de cada conjunto por intervalo de confianza para analiar sus resultados, tal y como se realizó en los apartados previos.

In [4]:
def encode_data_as_matrixes(train_texts:list, test_texts: list, max_n_words: int, sequence_len: int):
    '''
    Function that creates and trains a Keras tokenizer based on the provided
    train documents to encode the train and test data as numeric matrixes.

    Parameters
    ----------
    train_texts : list
        A list of strings with the train documents to encode.
    test_texts : list
        A list of strings with the test documents to encode.
    max_n_words : int
        Maximum number of words to keep within the LSTM memory
        based on computing the word frequency.
    sequence_len : int
        Maximum lenght of all sequences.
    
    Returns
    -------
    A
    '''
    # Create a tokenizer based on train texts
    tokenizer = Tokenizer(num_words=max_n_words)
    tokenizer.fit_on_texts(train_texts)

    # Transform each text into a numeric sequence
    train_sequences = tokenizer.texts_to_sequences(train_texts)

    # Transform each numeric sequence into a 2D vector
    train_matrix = pad_sequences(
        sequences=train_sequences, 
        maxlen=sequence_len)

    # Tokenize the test documents using the prior trained tokenizer
    test_sequences = tokenizer.texts_to_sequences(test_texts)

    # Transform each numeric sequence into a 2D vector
    test_matrix = pad_sequences(
        sequences=test_sequences,
        maxlen=sequence_len)
    
    return tokenizer, train_matrix, test_matrix

In [4]:
# Number of words to retain in the LSTM memory
MAX_N_WORDS = 1000

# Max number of tokens per numeric sequence
SEQUENCE_MAX_LEN = 100

# Path to the multilanguage embedding file
MULTILAN_EMBEDDINGS_FILE = '../en_es_embeddings/glove.twitter.27B.100d.txt'

# Filter the train documents by language
en_train_df = train_df[train_df['language'] == 'en']
en_train_texts = list(en_train_df['clean_text'].values)

# Filter the test documents by language
en_test_df = test_df[test_df['language'] == 'en']
en_test_texts = list(en_test_df['clean_text'].values)

# Convert processed train and test documents into matrixes
en_tokenizer, en_train_matrix, en_test_matrix = encode_data_as_matrixes(
    train_texts=en_train_texts,
    test_texts=en_test_texts,
    max_n_words=MAX_N_WORDS,
    sequence_len=SEQUENCE_MAX_LEN
)

# Load a pretrained BiLSTM model trained over English texts
en_bilstm_model = load_model('../models/en_lstm_models/en_bilstm_model_2L_128N_32BS.h5')

# Predict over the test dataset
en_test_df['task1_pred_probs'] = (en_bilstm_model.predict(en_test_matrix))
en_test_df['task1_pred_classes'] = (en_bilstm_model.predict(en_test_matrix) >= 0.5).astype('int32')

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

2023-04-25 16:33:34.938543: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-25 16:33:34.938560: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-25 16:33:34.938575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lidiasm): /proc/driver/nvidia/version does not exist
2023-04-25 16:33:34.938777: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


69/69 [==============================] - 7s 80ms/step


/tmp/ipykernel_28126/4124492836.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_probs'] = (en_bilstm_model.predict(en_test_matrix))


69/69 [==============================] - 7s 95ms/step
No. of false negatives: 398
No. of false positives: 246



/tmp/ipykernel_28126/4124492836.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_classes'] = (en_bilstm_model.predict(en_test_matrix) >= 0.5).astype('int32')


Del código propuesto anteriormente únicamente se ha modificado la función que calcula las predicciones para las perturbaciones basadas en una muestra de test ya que es requisito indispensable obtener una probabilidad para cada clase aunque el método de Keras únicamente devuelve la más elevada. Tras inspeccionar los resultados de los **falsos negativos** se pueden concluir que se repiten los fenómenos analizados en los modelos anteriores y que de nuevo se han visualizado ciertos **términos que son claramente sexistas pero no han sido tratados como tal**, lo que ha contribuido a cometer clasificaciones erroneas. 

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.2, 0.4), (-0.1, 0.2)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def encode_predict_lstm_model(sample: str):

        # Tokenize the test documents using the prior trained tokenizer
        sample_sequence = en_tokenizer.texts_to_sequences(sample)

        # Transform each numeric sequence into a 2D vector
        encoded_sample = pad_sequences(
            sequences=sample_sequence,
            maxlen=SEQUENCE_MAX_LEN)
            
        # Predict the probability for the max class
        one_class_pred_probs = en_bilstm_model.predict(encoded_sample)

        # Calculate the probability for each class
        biclass_pred_probs = [np.array([1-record[0], record[0]]) for record in one_class_pred_probs]
        return np.array(biclass_pred_probs)
    

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

En referencia a los **falsos positivos** existen una gran multitud de similitudes con respecto al análisis previo de muestras negativas erróneamente clasificadas, de nuevo he podido descubrir textos que aluden a otras temáticas y que han sido categorizados como positivos por contener palabras de género como "woman", "women", "men", etc. Sin embargo, un fenómeno muy destacable ha sido el hecho de encontrar documentos que contienen **negativas de términos sexistas** pero que han sido identificados como positivos puesto que el modelo no ha sido capaz de observar la expresión en su conjunto.

#### 5.1.4. Modelo LSTM español

A continuación se procede a replicar el procedimiento analítico anterior aunque sobre el mejor modelo LSTM encontrado para documentos en español. Tal y como se aprecia en los siguientes resultados, dispone de 392 falsos negativos y 210 falsos positivos.

In [5]:
# Number of words to retain in the LSTM memory
MAX_N_WORDS = 1000

# Max number of tokens per numeric sequence
SEQUENCE_MAX_LEN = 100

# Path to the multilanguage embedding file
MULTILAN_EMBEDDINGS_FILE = '../en_es_embeddings/glove.twitter.27B.100d.txt'

# Filter the train documents by language
es_train_df = train_df[train_df['language'] == 'es']
es_train_texts = list(es_train_df['clean_text'].values)

# Filter the test documents by language
es_test_df = test_df[test_df['language'] == 'es']
es_test_texts = list(es_test_df['clean_text'].values)

# Convert processed train and test documents into matrixes
es_tokenizer, es_train_matrix, es_test_matrix = encode_data_as_matrixes(
    train_texts=es_train_texts,
    test_texts=es_test_texts,
    max_n_words=MAX_N_WORDS,
    sequence_len=SEQUENCE_MAX_LEN
)

# Load a pretrained LSTM model trained over Spanish texts
es_lstm_model = load_model('../models/es_lstm_models/es_lstm_3L_1286432N_16BS.h5')

# Predict over the test dataset
es_test_df['task1_pred_probs'] = (es_lstm_model.predict(es_test_matrix))
es_test_df['task1_pred_classes'] = (es_lstm_model.predict(es_test_matrix) >= 0.5).astype('int32')

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

2023-04-26 08:50:56.723333: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-26 08:50:56.723594: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-26 08:50:56.723612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lidiasm): /proc/driver/nvidia/version does not exist
2023-04-26 08:50:56.724122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 3/67 [>.............................] - ETA: 2s

/tmp/ipykernel_5295/1679225747.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_probs'] = (es_lstm_model.predict(es_test_matrix))


67/67 [==============================] - 2s 34ms/step
No. of false negatives: 392
No. of false positives: 210



/tmp/ipykernel_5295/1679225747.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_classes'] = (es_lstm_model.predict(es_test_matrix) >= 0.5).astype('int32')


Los documentos submuestreados para una inspección visual y una explicabilidad local de los **falsos negativos** han apuntado hacia nuevos fenómenos explicativos del pésimo comportamiento del modelo. En primer lugar existen textos que no contienen terminología sexista pero sí que pertenecen a esta clase. Sin embargo, desde una perspectiva técnica y automática no contienen pistas que hagan pensar que se trata de muestras positivas, para ello sería necesario un **mayor nivel de entendimiento del lenguaje natural** por parte del modelo que se asimilase al del humano. Por otro lado también son ampliamente utilizados **conceptos informales y casuales**, que no se encuentran comunmente en los diccionarios oficiales de idiomas, y que por tanto no son reconocidos automáticamente provocando una pérdida de información vital.

No obstante en el estudio de los **falsos positivos** las pesquisas obtenidas son las ya conocidas hasta el momento con documentos con terminología propia de la temática considerada como sexista liderando los fallos cometidos, así como reivindicaciones con expresiones sexistas pero con un significado positivo.

#### 5.1.5. Modelo BERT inglés

Para finalizar la generación de explicaciones locales en base a muestras individuales en este apartado se lleva a cabo el procedimiento conocido hasta el momento en el que se analizan los falsos negativos y positivos del mejor modelo BERT producido para documentos ingleses. En el siguiente resultado se aprecia la existencia de 267 falsos negativos y 309 falsos positivos, de los cuales se muestrean diez instancias por cada conjunto e intervalo de confianza establecido.

In [3]:
def tokenize_texts(texts: list, labels: list, bert_tokenizer: BertTokenizer):
    '''
    A function that tokenizes the provided list of texts using a pre-trained
    Bert tokenizer to encode the documents to get them ready for BERT models.

    Parameters
    ----------
    texts : list
        A list of strings with the documents to encode as BERT tokens.
    labels : int
        A list of integers with the class labels to encode as a Numpy array.
    bert_tokenizer : BertTokenizer
        A pre-trained BERT tokenizer to use to encode the documents for BERT models.
    
    Returns
    -------
    A dictionary with the tokenized texts, the created attention
    masks for the BERT models as well as the transformed class labels
    within a Numpy array.
    '''
    tokenized_texts = []
    attention_masks = []

    # Tokenize the provided texts using BERT tokenizer
    for doc in texts:
        bert_inp = bert_tokenizer.encode_plus(
            doc,
            add_special_tokens=True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True)

        # Get the tokenized texts along with the attention masks
        tokenized_texts.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    return {
        'tokenized_texts': np.asarray(tokenized_texts),
        'attention_masks': np.asarray(attention_masks),
        'labels': np.asarray(labels),
    }

In [4]:
# Load a pre-trained BertTokenizer for English texts
BERT_TOKENIZER_OBJ = BertTokenizer.from_pretrained('bert-base-uncased')

# Filter train and test texts by language
en_test_df = test_df[test_df['language'] == 'en']

# Tokenize the test documents
en_bert_encoded_test_data = tokenize_texts(
    texts=list(en_test_df['clean_text'].values),
    labels=list(en_test_df['task1'].values),
    bert_tokenizer=BERT_TOKENIZER_OBJ)

# Load a trained BERT classifier
en_bert_model = TFBertForSequenceClassification.from_pretrained('../models/en_bert_models/en_bert_model_synN2X2_2e5LR_8BS')

# Create the predictions over the test dataset
en_test_preds = en_bert_model.predict(
[
    en_bert_encoded_test_data['tokenized_texts'], 
    en_bert_encoded_test_data['attention_masks']
])
# Predict over the test dataset
en_test_probs = [max(value) for value in np.asarray(tf.nn.sigmoid(en_test_preds.logits))]

# Insert the predicted classes and probabilities into the test dataset
en_test_df['task1_pred_classes'] = np.argmax(en_test_preds.logits, axis=1)
en_test_df['task1_pred_probs'] = [max(value) for value in np.asarray(tf.nn.sigmoid(en_test_preds.logits))]

# Get the indexes of the false negatives and positives
en_fn_df = en_test_df[(en_test_df['task1'] == 1) & (en_test_df['task1_pred_classes'] == 0)]
en_fp_df = en_test_df[(en_test_df['task1'] == 0) & (en_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {en_fn_df.shape[0]}')
print(f'No. of false positives: {en_fp_df.shape[0]}\n')

/home/lidia/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2023-04-27 08:58:48.152237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-27 08:58:48.152255: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-27 08:58:48.152271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.c

69/69 [==============================] - 154s 2s/step
No. of false negatives: 267
No. of false positives: 309



/tmp/ipykernel_39339/2275121338.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_classes'] = np.argmax(en_test_preds.logits, axis=1)
/tmp/ipykernel_39339/2275121338.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_test_df['task1_pred_probs'] = [max(value) for value in np.asarray(tf.nn.sigmoid(en_test_preds.logits))]


Al modificar la arquitectura es requisito fundamental la adaptación de la función que se le proporciona a LIME para calcular la predicción de cada perturbación generada. Según el análisis efectuado acerca de los **falsos negativos** visualizados anteriormente se han podido obtener las siguientes conclusiones:

* La principal diferencia destacable con respecto a las explicaciones de los modelos previos reside en que el modelo BERT es capaz de **categorizar un término como sexista o no sexista dependiendo del contexto**, ya que en algunos documentos el concepto "women" aparece como positivo mientras que en otros pertenece a la clase contraria. Por lo tanto esta arquitectura manifiesta un mayor potencial de comprensión del lenguaje natural que los clasificadores conocidos hasta el momento.

* Asimismo también se observa el fenómeno descubierto durante el análisis exploratorio de datos en el que existen muestras que pueden resultar confusas para su identificación automática por su alto **contenido en terminología y expresiones positivas aunque con un significado profundamente sexista**. Este hecho recuerda el trabajo que aún queda por hacer para mejorar el entendimiento del lenguaje natural por parte de las máquinas introduciendo capacidades propias como la ironía.

* Finalmente conforme se desciende hacia los intervalos de confianza inferiores los documentos que se visualizan resultan confusos hasta para un ser humano puesto que su **modo de redacción es tan pobre** que ni yo misma he reconocido sus significados. Como consecuencia considero que este tipo de errores son completamente normales e incluso no deberían ser contabilizados por su pésima calidad gramatical y semántica.

In [ ]:
# Create a LIME explainer object
lime_explainer = LimeTextExplainer(class_names={
    0: 'non-sexist',
    1: 'sexist'
})

for pair in [(0.4, 0.6), (0.2, 0.4), (-0.1, 0.2)]:
    # Filter the false negatives by confidence interval
    en_fn_subdf = en_fn_df[(en_fn_df['task1_pred_probs'] >= pair[0]) & \
        (en_fn_df['task1_pred_probs'] < pair[1])]
    en_fn_subdf_texts = list(en_fn_subdf['clean_text'].values)

    # Choose 10 distinct samples randomly
    chosen_fn_texts = []
    for iter in range(0, 10):
        fn_index = randrange(len(en_fn_subdf_texts))
        while (fn_index in chosen_fn_texts):
            fn_index = randrange(len(en_fn_subdf_texts))

        chosen_fn_texts.append(fn_index)

    # Create explanations for each sample
    def encode_predict_bert_model(samples: str):
        tokenized_texts = []
        attention_masks = []
        
        # Encode each sample in a BERT way
        for text in samples:
            encoded_sample = BERT_TOKENIZER_OBJ.encode_plus(
                text,
                add_special_tokens=True,
                max_length=64,
                pad_to_max_length=True,
                return_attention_mask=True)
            
            # Get the tokenized texts along with the attention masks
            tokenized_texts.append(encoded_sample['input_ids'])
            attention_masks.append(encoded_sample['attention_mask'])

        # Create the predictions over the test dataset
        en_test_preds = en_bert_model.predict(
        [
            np.asarray(tokenized_texts), 
            np.asarray(attention_masks)
        ])
        
        # Get the probabilities for each test prediction
        return np.asarray(tf.nn.sigmoid(en_test_preds.logits))    
        

    for indx in chosen_fn_texts:
        # Create a single explanation for a single test sample
        lime_explanation = lime_explainer.explain_instance(
            en_fn_subdf_texts[indx], 
            predict_en_lrm_model).show_in_notebook(text=True)

Tras el estudio de los **falsos positivos** de este mismo modelo inglés se ha podido observar un fenómeno común al resto de modelos, que si bien las arquitecturas BERT han demostrado producir un contexto más preciso y de mayor calidad, siguen fallando en algunos documentos que representan **reivindicaciones contra la violencia machista** aunque se encuentran compuestas mayoritariamente por términos negativos que producen su clasificación errónea como muestras sexistas. Adicionalmente de un modo similar a los ejemplos irónicos el clasificador también comete errores al analizar documentos con **expresiones hechas** como *"hacerse la rubia"* que son tratados como sexistas aunque el tópico tratado no está relacionado remotamente.

#### 5.1.6. Modelo BERT español

Finalizando este apartado de explicaciones locales por último se replica el procedimiento aplicado actualmente aunque empleando el mejor modelo BERT que se ha conseguido para textos españoles. En los siguientes resultados se puede observar que se caracteriza por tener 300 falsos negativos y 184 falsos positivos.

In [4]:
# Load a pre-trained BertTokenizer for Spanish texts
BERT_TOKENIZER_OBJ = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

# Filter train and test texts by language
es_test_df = test_df[test_df['language'] == 'es']

# Tokenize the test documents
es_bert_encoded_test_data = tokenize_texts(
    texts=list(es_test_df['clean_text'].values),
    labels=list(es_test_df['task1'].values),
    bert_tokenizer=BERT_TOKENIZER_OBJ)

# Load a trained BERT classifier
es_bert_model = TFBertForSequenceClassification.from_pretrained('../models/es_bert_models/es_bert_model_synN1X3_1e5LR_8BS')

# Create the predictions over the test dataset
es_test_preds = es_bert_model.predict(
[
    es_bert_encoded_test_data['tokenized_texts'], 
    es_bert_encoded_test_data['attention_masks']
])

# Insert the predicted classes and probabilities into the test dataset
es_test_df['task1_pred_classes'] = np.argmax(es_test_preds.logits, axis=1)
es_test_df['task1_pred_probs'] = [max(value) for value in np.asarray(tf.nn.sigmoid(es_test_preds.logits))]

# Get the indexes of the false negatives and positives
es_fn_df = es_test_df[(es_test_df['task1'] == 1) & (es_test_df['task1_pred_classes'] == 0)]
es_fp_df = es_test_df[(es_test_df['task1'] == 0) & (es_test_df['task1_pred_classes'] == 1)]

print(f'No. of false negatives: {es_fn_df.shape[0]}')
print(f'No. of false positives: {es_fp_df.shape[0]}\n')

/home/lidia/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2023-04-27 14:17:54.305896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-27 14:17:54.305914: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-27 14:17:54.305928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.c

67/67 [==============================] - 149s 2s/step
No. of false negatives: 300
No. of false positives: 184



/tmp/ipykernel_48729/380653055.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_classes'] = np.argmax(es_test_preds.logits, axis=1)
/tmp/ipykernel_48729/380653055.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  es_test_df['task1_pred_probs'] = [max(value) for value in np.asarray(tf.nn.sigmoid(es_test_preds.logits))]


Mientras que en el estudio de los **falsos negativos** se han vuelto a visualizar las conclusiones extraídas en modelos anteriores, como la aparición de **hashtags** que dificultan la evaluación del grado de sexismo de los términos y los documentos repletos de conceptos y expresiones positivas con significados sumamente negativos.

Por otro lado analizando los **falsos positivos** me he percatado de que la mayor parte de documentos pertenecientes a este conjunto se caracterizan también por tener un **contenido difuso**, apenas comprensible por una persona, y acerca de una amplia diversidad de temáticas, como noticias del corazón, mensajes homofóbicos, etc.

#### 5.1.6. Conclusiones tras aplicar LIME

* En primer lugar cabe destacar el hecho de que LIME únicamente analiza términos individuales en lugar de conjuntos de palabras o expresiones que podría ser más útil ya que conceptos por sí solos pueden no ser sexistas a no ser que se encuentren acompañados de otros que le otorguen ese significado, como violencia de género.

* En los modelos de Regresión Logística y LSTM se ha podido descubrir que **terminología que debía ser neutra**, como pronombres, determinantes, preposiciones, se les ha asignado un valor no sexista que contrarresta, por su elevado volumen de aparación, a los conceptos que sí lo son provocando una multitud de muestras erróneamente clasificadas. Igualmente ocurre con **palabras comunes a este tópico**, como mujeres, marido, hombres, que son tratadas como sexistas de forma individual lo que conlleva también un considerable número de fallos. Sin embargo, las arquitecturas **BERT sí son capaces de asignar una clase u otra a un mismo término dependiendo del contexto**, lo que demuestra una mejor habilidad de predicción y por ende unos mejores resultados en las métricas de validación.

* Otra muestra más del trabajo que queda por realizar de forma general a todas las arquitecturas existentes para aumentar el nivel de comprensión del lenguaje natural por modelos automáticos reside en aquellos documentos que muestran un **elevado grado de ironía, expresiones hechas o negaciones** que son categorizados dentro de la clase positiva.

* Una clara diferencia entre los errores cometidos por los modelos más sencillos en comparación con los propios de las arquitecturas BERT reside en que estos últimos cuentan con **documentos de una pésima calidad ortográfica, gramatical y semántica** que incluso dificulta su identificación a una persona humana. Por lo tanto se trata de muestras de test que podrían ser no consideradas debido al maltrecho contenido que presentan.

* Finalmente se ha descubierto que **tanto los hashtags como los errores gramaticales producen una falta de información**, severa en ciertos documentos, que provocan la pérdida de información liderando hacia una clasificación errónea de las muestras. Ambos fenómenos se encuentran más presentes en los ejemplos erróneamente clasificados bajo **intervalos moderados y bajos de confianza**, puesto que en estos casos el modelo apenas dispone de términos útiles en los que fundamentar su decisión y por ende se muestra más inseguro.

### 5.2. Contrafactuales

Las explicaciones contrafactuales consisten en producir perturbaciones a partir de una muestra de ejemplo con la que descubrir las causas que inducen al modelo a cambiar su comportamiento. Dentro del área del Procesamiento del Lenguaje Natural esta técnica permite estudiar **qué cambios mínimos son necesarios para modificar la predicción de un modelo en relación a un documento concreto**. Con esta idea se ha aplicado a este proyecto con el objetivo de analizar qué modificaciones serían requeridas para que los falsos negativos y positivos pudieran ser categorizados correctamente reduciendo el número de cambios que se le deben efectuar a cada texto. Si bien pueden ser generados manualmente o automáticamente, se ha optado por la segunda vía con la que maximizar su eficiencia y volumen de muestras. Sin embargo, el volumen de librerías en Python que faciliten este procedimiento es sumamente escaso y de las que he probado no he conseguido que ninguna pudiese siquiera ejecutar el ejemplo propuesto en sus respectivas páginas de GitHub.

* [MICE](https://github.com/allenai/mice): requiere la codificación de nuevas clases y el entrenamiento de nuevos modelos predictores y editores adaptados al conjunto de datos EXIST.
* [SEDC](https://github.com/yramon/edc): error de tipo de dato.
* [Polyjuice](https://github.com/tongshuangwu/polyjuice): error por la inadaptación a un cambio realizado en la librería *transformers* de la que depende para usar el modelo. Para usar este transformer es necesario comunicarle los lugares en los que se desea realizar uno o diversos tipos de cambios, por lo que se trataría de un proceso semi-automático puesto que debe estar personalizado para cada muestra.
* [DICE](https://github.com/interpretml/DiCE): necesita características numéricas, no documentos.
* [MLXTEND](http://rasbt.github.io/mlxtend/user_guide/evaluate/create_counterfactual/): necesita características numéricas, no documentos.

Al revisar las distintas operaciones que llevan a cabo para elaborar nuevos documentos sintéticos, he pensado que podría realizar una implementación simplificada de aquellas que pueden ser más efectivas y clarificantes:

* **Eliminación de palabras**. Se genera un nuevo ejemplo tras eliminar un único término o dos conceptos consecutivos tras la iteración de todos los vocablos de un documento base.

* **Negación de verbos**. Se genera una nueva muestra transformando sus verbos de afirmativos a negativos y viceversa con el objetivo de representar el significado opuesto de un texto base. Este método será semiautomático combinando el uso de GPT-3 bajo mi supervisión puesto que existe un considerable volumen de documentos difíciles de procesar por modelos inteligentes.

La experimentación de esta segunda técnica se pretende que sea idéntica a la anterior muestreando **diez ejemplos aleatoriamente de cada intervalo de confianza** para el análisis de falsos negativos y positivos en los mejores modelos obtenidos tanto en inglés como en español. 

In [ ]:
def create_counterfactuals_by_removing_words_per_interval(conf_intervs, test_df):
    '''
    Function that creates counterfactuals by removing one word each time taking
    ten texts randomly from the provided set of test documents as seeds. This
    procedure could be repeated for every confidence interval.

    Parameters
    ----------
    conf_intervs : list
        A list of tuples with the confidence intervals to
        filter the test documents and create counterfactuals.
    test_df : Pandas dataframe
        A dataframe with the test documents to use as seeds
        to create counterfactuals and the predicted probabilities
        to filter them by confidence intervals.
    
    Returns
    -------
    A dictionary with the test text seed as key and the 
    created counterfactual as value.
    '''
    seed_test_indexes = []
    counterfactual_texts = {}

    for pair in conf_intervs:
        # Filter the test samples by confidence interval
        sub_test_df = test_df[(test_df['task1_pred_probs'] >= pair[0]) & \
            (test_df['task1_pred_probs'] < pair[1])]
        test_texts = list(sub_test_df['clean_text'].values)

        # Choose 10 test texts randomly 
        for iter in range(0, 10 if len(test_texts) >= 10 else len(test_texts)):
            index = randrange(len(test_texts))
            while (index in seed_test_indexes):
                index = randrange(len(test_texts))
            
            # Create counterfactuals by removing words
            seed_test_indexes.append(index)
            seed_text = test_texts[index]
            counterfactual_texts[seed_text] = create_new_texts_by_removing_words(seed_text)
            
    return counterfactual_texts

In [ ]:
def create_counterfactuals_by_changing_verbs_gpt3(conf_intervs, test_df, lang):
    '''
    Function that creates counterfactuals by using GPT-3 to change the meaning
    of the verbs converting the positive ones to negative and viceversa. To do
    that the function chooses ten texts randomly from the provided set of test 
    documents as seeds. This procedure could be repeated for every 
    confidence interval.

    Parameters
    ----------
    conf_intervs : list
        A list of tuples with the confidence intervals to
        filter the test documents and create counterfactuals.
    test_df : Pandas dataframe
        A dataframe with the test documents to use as seeds
        to create counterfactuals and the predicted probabilities
        to filter them by confidence intervals.
    
    Returns
    -------
    A dictionary with the test text seed as key and the 
    created counterfactual as value.
    '''
    seed_test_indexes = []
    counterfactual_texts = {}

    for pair in conf_intervs:
        # Filter the test samples by confidence interval
        sub_test_df = test_df[(test_df['task1_pred_probs'] >= pair[0]) & \
            (test_df['task1_pred_probs'] < pair[1])]
        test_texts = list(sub_test_df['clean_text'].values)

        # Choose 10 test texts randomly 
        for iter in range(0, 10 if len(test_texts) >= 10 else len(test_texts)):
            index = randrange(len(test_texts))
            while (index in seed_test_indexes):
                index = randrange(len(test_texts))
            
            # Create counterfactuals by removing words
            seed_test_indexes.append(index)
            seed_text = test_texts[index]
            counterfactual_texts[seed_text] = generate_text_using_gpt3(seed_text, lang)

    return counterfactual_texts

#### 5.2.1. Modelo inglés de Regresión Logística

En relación a **falsos negativos** se ha podido observar que únicamente seis muestras erróneamente clasificadas se han podido corregir **eliminando ciertos términos tales como preposiciones, artículos, determinantes, adverbios**, puesto que, tal y como se comentó en los análisis de rendimiento de este modelo con LIME, son etiquetados como no sexistas y pueden contrarrestar el grado de los conceptos que sí son sexistas alterando así su categorización. Por otro lado negando los verbos existentes en estos falsos negativos solo se ha conseguido clasificar correctamente una única muestra motivado por la transformación de prácticamente la totalidad de sus verbos que han contribuido a aumentar la probabilidad de la clase positiva.

Por otro lado en la generación y análisis de contrafactuales aplicados a **falsos positivos** se ha visualizado una **tendencia similar** en sendas operaciones, destacando también la supresión de vocablos adicionales como palabras típicas de esta temática como terminología relativa a ambos géneros, insultos sexuales, etc.

Finalmente cabe destacar que todos los documentos cuyas clases han podido ser corregidas gracias a la generacón de contrafactuales pertenencen al **intervalo moderado de confianza 40%-60%**, un fenómeno que ya se pudo intuir en los notebooks de análisis de modelos puesto que en la mayoría de errores cometidos los sistemas demostraban una seguridad demasiado elevada como para poder contrarrestarla con modificaciones mínimas en el contenido de los textos.

#### 5.2.2. Modelo español de Regresión Logística

Tras el análisis de los contrafactuales generados para los **falsos negativos** se ha podido evidenciar una **inclinación similar a la anterior** en la que las muestras corregidas pertenecen al **intervalo moderado de confianza**, mientras que los textos modificados utilizando la primera operación de eliminar términos individuales han desembocado en clasificaciones correctas gracias a la supresión de preposiciones, adverbios, etc. Asimismo, de nuevo empleando la segunda función de **negación de verbos no se han obtenido resultados** positivos por lo que ningún ejemplo erróneamente categorizado ha podido ser reconducido hacia su clase correcta.

Al contrario ha ocurrido con los **falsos positivos** en los que sí se han visualizado correcciones de muestras incorrectamente clasificadas **pertenecientes a todos los intervalos de confianza**, aunque siendo mayoritariamente el moderado de entre 40%-60%. En este caso, al igual que sucedía con el modelo inglés, la **eliminación de insultos y palabras tanto sexuales como típicas de la temática**, como mujer, han sido las claves para redirigir este comportamiento. No obstante, tampoco se han obtenido correcciones en el uso de la negación de verbos.

#### 5.2.3. Modelo LSTM inglés

Si bien en el análisis de **falsos negativos** se han vuelto a percibir los mismos fenómenos conocidos hasta el momento en la generación y comprobación de contrafactuales eliminando términos de los documentos, la sorpresa procede de la segunda operación en la que se utiliza GPT-3 para negar los verbos de los textos. Aunque únicamente se han podido corregir dos muestras pertenecientes a los dos primeros intervalos de mayor confianza, además de la detección y negación de verbos GPT-3 también ha sido capaz de **corregir errores ortográficos, convertir hashtags en palabras individuales e incluso parafrasear** ciertos documentos para conseguir el significado opuesto que se le pedía como tarea. 

En referencia al estudio de **falsos positivos** de nuevo se han visualizado unas tendencias similares a las anteriores en las que **eliminando términos sexuales y típicos de la temática** se han podido clasificar correctamente un elevado volumen de muestras erróneamente categorizadas como positivas. Como en el caso de los falsos positivos el único ejemplo que se ha corregido negando los verbos existentes en su contenido ha sido gracias al **parafraseado** del texto que le ha proporcionado un significado más claro para el modelo LSTM inglés.

#### 5.2.4. Modelo LSTM español

Durante el análisis realizado sobre los **falsos negativos** se ha conseguido un elevado grado de corrección de muestras tanto eliminando términos de su contenido como modificando el significado de sus verbos para ser opuestos a los oroginales en prácticamente todos los intervalos de confianza establecidos. Especialmente destaca la capacidad de GPT-3 para **reemplazar palabras gramaticalmente erróneas** por las correctas sin modificar el sentido del documento, fenómeno que no se pudo conseguir con diversas librerías de Python experimentadas, la **conversión de hashtags en sus correspondientes textos** dentro del documento general y el **parafraseo** de ciertos verbos más complicados de negar con el objetivo de que sea lo más natural posible.

En los **falsos positivos** se ha concluido pesquisas similares a las anteriores en las que eliminando términos sexuales y de género comunes a la temática abordada se consigue corregir documentos erróneamente clasificados pertenecientes a todos los intervalos de confianza, aunque especialmente al moderado de entre 40-60%. Por el contrario la negación de verbos no ha proporcionado tan buenos resultados como en el caso previo siendo sumamente notables los cambios realizados sobre los **documentos convirtiéndolos prácticamente en nuevos textos sin ninguna relación entre sí**. Así se ha puesto de manifiesto la necesidad de establecer esta tarea como **semi-automática** bajo supervisión de un humano con el fin de clarificar la causa que produce una re-clasificación de una muestra concreta.

#### 5.2.5. Modelo BERT inglés

En el estudio de los **falsos negativos** destaca la **imposibilidad de corregir ninguna muestra perteneciente al intervalo de confianza más elevado** entre el 80% y 100%, lo cual ya se intuía en los análisis de resultados del clasificador BERT inglés puesto que se presuponía complicado el poder contradecir las decisiones que tomaba con tanta seguridad. Sin embargo, la tendencia observada es similar a los modelos previos en los que los ejemplos corregidos han sido motivados por la eliminación de todo tipo de términos desde sustantivos, determinantes, articulos, preposiciones, hashtags, entre otros. Por otro lado aplicando la segunda operación disponible capaz de negar el significado de los verbos únicamente se ha conseguido redireccionar dos datos a partir de la **división de hashtags en sus términos individuales** para ser codificados por los embeddings así como por el beneficio de haber modificado el significado del documento sustituyendo los verbos existentes por sus opuestos.

En el caso de los **falsos positivos** sí que aparecen muestras correctamente clasificadas en todos los intervalos a partir de la supresión de palabras típicas de los géneros, sexistas y otro tipo de terminología. Ocurre una situación contraria al contraponer los verbos existentes siendo menor el parafraseo realizado por GPT-3 debido a la mayor calidad que caracteriza a los textos, estando presente solamente en una minoría.

#### 5.2.6. Modelo BERT español

Mientras que en el análisis de **falsos negativos** se han encontrado correcciones de muestras pertenecientes a todos los intervalos disponibles, desde el más elevado hasta el moderado, a partir de la supresión de diferente terminología como articulos, determinantes, pronombres, etc, también se han conseguido resultados beneficiosos aplicando esta primera operación al **eliminar negaciones de verbos y adjetivos que han propiciado la clasificación correcta** de lagunos ejemplos. Sin embargo, existen otros fenómenos ya visualizados en anteriores notebooks en los que se contrarresta el grado de sexismo otorgado a un documento gracias al tratamiento no sexista de términos que no tienen relación con la temática, como la palabra *"dinosaurio"*. Por otro lado en el segundo procedimiento efectuado únicamente se han conseguido redireccionar cinco muestras debido a un **potente parafraseo de GPT-3 que ha clarificado el sentido y significado de los documentos**, así como la negación de los principales verbos que ha facilitado su entendimiento opuesto para su categorización contraria.

En relación a los **falsos positivos** de nuevo han sido generados a partir de la eliminación de conceptos sexuales, palabras que hacen referencia a ambos géneros, determinantes, artículos, términos con connotaciones negativas generales *"esclavitud"*, e incluso palabras comunmente utilizadas como **"piropos" que han sido empleadas en un contexto distinto**. En el caso de la negación de los verbos almacenados en los documentos de este subconjunto se ha apreciado la corrección de un mínimo volumen de muestras a partir de un **elevado nivel de correcciones gramaticales, divisón de hashtags y parafraseo** proporcionado por el modelo GPT-3.

#### 5.2.7. Conclusiones tras generar contrafactuales

* El primer hecho destacable es que existe una mayor probabilidad de generar contrafactuales que puedan corregir muestras erróneamente clasificadas eliminando cada término en una iteración distinta puesto que se producen un mayor número de ejemplos con los que probar el comportamiento del modelo.

* La segunda conclusión general reside en la **corrección de un mayor volumen de muestras pertenecientes a los intervalos de confianza moderados-bajos**, mientras que ha sido sumamente complicado y prácticamente imposible, en algunos casos, la redirección de falsos negativos y positivos que se encuentran en intervalos de confianza elevados. Como ya se predijo en anteriores notebooks, los modelos demostraban una seguridad bastante superior al cometer estos errores por lo que su redirección a las categorías correctas ya se intuía sumamente difícil.

* Existe una tendencia prácticamente general a todos los modelos en relación a los **falsos negativos** puesto que se ha percibido cómo se han redireccionado hacia su clase positiva al eliminar palabras de parada como **artículos, determinantes, pronombres, preposiciones**. Sin embargo también ha influido otro tipo de terminología como sustantivos y verbos de cualquier naturaleza, que son tratados como no sexistas y contrarrestan el valor asociado a los conceptos que sí son sexistas. Por lo que una posible solución parcial podría ser suprimir este tipo de vocablos y así minimizar este fenómeno indeseado.

* Por otro lado en los análisis de **falsos positivos** se ha descubierto un patrón de mejora al suprimir **palabras sexistas, conceptos típicos de ambos géneros y términos con connotaciones sexuales**. Así se ha reducido el grado de sexismo computado y se han podido clasificar algunos ejemplos en su categoría correcta. No obstante, esta operación no parece ser una solución viable puesto que a priori no se conoce si una nueva muestra es o no sexista y el hecho de eliminar estas expresiones podría aumentar considerablemente el número de falsos negativos. El problema reside en que en ciertos documentos determinar el contexto en el que se utiliza esta terminología es muy complicada, ya que muchos de ellos carecen de sentido común, no están bien redactados y contienen numerosas faltas de ortografía y hashtags que dificultan su codificación.

* El **uso de GPT-3** para convertir los verbos a su significado opuesto ha sido muy beneficioso en tanto en cuanto se han podido investigar diferentes observaciones relevantes, como su capacidad de parafrasear un documento otorgándole un **hilo conversacional correcto** y comprensible por un sistema automático, la **división en términos independientes de hashtags** que favorecen su codificación y la **corrección de conceptos ortográficamente y gramaticalmente erróneos** que de nuevo favorecen un entendimiento más detallado de su significado. No obstante, en ocasiones GPT-3 puede llegar a realizar demasiadas modificaciones que desembocan en la pérdida prácticamente total de información e incluso de relación con respecto al texto original, por lo que ya no se trataría de un contrafactual sino de una muestra nueva. Como consecuencia **esta tarea debe ser supervisada por una persona** que compruebe el número de cambios realizados y su impacto tomando en consideración el ejemplo establecido.